In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import TFResNetModel, AutoImageProcessor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns

2025-08-18 19:41:16.830312: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755546076.859270      98 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755546076.867654      98 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
handcrafted_features_path = '/kaggle/input/features/features.csv'  # Update with actual input dataset name
metadata_path = '/kaggle/input/metadata-csv/HAM10000_metadata.csv'  # Update with actual
image_part1_path = '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1'
image_part2_path = '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2'

In [3]:
# Load ResNet50 from Hugging Face
processor = AutoImageProcessor.from_pretrained('microsoft/resnet-50')
base_model = TFResNetModel.from_pretrained('microsoft/resnet-50')

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
2025-08-18 19:41:35.192800: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFResNetModel: ['resnet.encoder.stages.1.layers.1.layer.1.normalization.num_batches_tracked', 'resnet.encoder.stages.1.layers.2.layer.1.normalization.num_batches_tracked', 'resnet.encoder.stages.3.layers.2.layer.0.normalization.num_batches_tracked', 'resnet.encoder.stages.2.layers.1.layer.2.normalization.num_batches_tracked', 'resnet.encoder.stages.0.layers.0.layer.1.normalization.num_batches_tracked', 'resn

In [4]:
# Load metadata for labels
metadata_df = pd.read_csv(metadata_path)
metadata_df['label'] = metadata_df['dx'].apply(lambda x: 1 if x == 'mel' else 0)

In [ ]:
# Prepare dataset for fine-tuning (binary classification)
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(metadata_df, test_size=0.2, stratify=metadata_df['label'], random_state=42)

# Create TF datasets for fine-tuning
def load_image(file_path, label):
    def process_image_fn(file_path):
        # Load and decode image
        img = tf.io.read_file(file_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [224, 224])
        img = tf.cast(img, tf.uint8)
        
        # Convert to numpy for the processor
        img_np = img.numpy()
        
        # Use processor (returns numpy array when return_tensors='np')
        processed = processor(images=img_np, return_tensors='np')
        return processed['pixel_values'][0].astype('float32')
    
    # Use tf.py_function to handle the mixed operations
    pixel_values = tf.py_function(
        process_image_fn,
        [file_path],
        tf.float32
    )
    
    # ResNet processor outputs to set shape accordingly
    pixel_values.set_shape([3, 224, 224])
    
    return pixel_values, label

In [6]:
# Get full image paths
train_df['file_path'] = train_df['image_id'].apply(lambda x: os.path.join(image_part1_path if os.path.exists(os.path.join(image_part1_path, x + '.jpg')) else image_part2_path, x + '.jpg'))
val_df['file_path'] = val_df['image_id'].apply(lambda x: os.path.join(image_part1_path if os.path.exists(os.path.join(image_part1_path, x + '.jpg')) else image_part2_path, x + '.jpg'))

train_dataset = tf.data.Dataset.from_tensor_slices((train_df['file_path'], train_df['label']))
train_dataset = train_dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((val_df['file_path'], val_df['label']))
val_dataset = val_dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Add classification head for fine-tuning
inputs = tf.keras.Input(shape=(3, 224, 224))  # Match processor output shape
x = base_model(inputs, training=True).last_hidden_state  # Use base_model directly, not base_model.resnet
# For ResNet, last_hidden_state should be pooled
x = tf.keras.layers.GlobalAveragePooling2D(data_format='channels_first')(x)  # Specify channels_first
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
fine_tune_model = tf.keras.Model(inputs, outputs)

In [ ]:
# Compile and fine-tune
fine_tune_model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])
fine_tune_model.fit(train_dataset, validation_data=val_dataset, epochs=5)  # Adjust epochs as needed

# Use fine-tuned model for extraction
model = fine_tune_model  

In [ ]:
model = fine_tune_model

model = tf.keras.models.load_model(
    '/kaggle/input/fine-tuned/tensorflow2/default/1/fine_tuned_resnet50.h5',
    custom_objects={'TFResNetModel': TFResNetModel}  # Register the custom layer
)
print("Model loaded successfully!")

Model loaded successfully!


In [ ]:
# Feature Extraction Function (using the fine-tuned model)
def extract_features(image_path):
    img = Image.open(image_path).resize((224, 224))
    img_array = np.array(img)
    inputs = processor(img_array, return_tensors='tf')
    
    # Get features from the layer before the final classification layer
    feature_extractor = tf.keras.Model(inputs=model.input, outputs=model.layers[-2].output)
    features = feature_extractor(inputs['pixel_values'], training=False)
    features = features.numpy().flatten()
    return features

In [10]:
# Extract features for all images
feature_vectors = []
image_ids = []
for dir_path in [image_part1_path, image_part2_path]:
    for img_file in tqdm(os.listdir(dir_path)):
        if img_file.endswith('.jpg'):
            img_path = os.path.join(dir_path, img_file)
            features = extract_features(img_path)
            feature_vectors.append(features)
            image_ids.append(img_file.replace('.jpg', ''))
            
# Create DataFrame and save raw features
features_df = pd.DataFrame(feature_vectors, index=image_ids)
features_df.to_csv('/kaggle/working/resnet50_finetuned_raw_features.csv')
print("Raw features saved to '/kaggle/working/resnet50_finetuned_raw_features.csv'")

100%|██████████| 5015/5015 [30:22<00:00,  2.75it/s]


Raw features saved to '/kaggle/working/resnet50_finetuned_raw_features.csv'


In [12]:
# Load hand-crafted features for redundancy check
handcrafted_df = pd.read_csv(handcrafted_features_path, index_col='image_id')

common_ids = list(set(features_df.index) & set(handcrafted_df.index))
dl_df = features_df.loc[common_ids]
hand_df = handcrafted_df.loc[common_ids]

# Convert columns to strings to avoid mixed type issues
dl_df.columns = dl_df.columns.astype(str)
hand_df.columns = hand_df.columns.astype(str)

concat_df = pd.concat([dl_df, hand_df], axis=1)
corr_matrix = concat_df.corr().abs()
corr_between = corr_matrix.iloc[:len(dl_df.columns), len(dl_df.columns):]

to_drop_dl = [dl_col for dl_col in dl_df.columns if (corr_between.loc[dl_col] > 0.7).any()]  # Use loc for row access
print(f"Redundant DL features to drop: {to_drop_dl}")

# Remove from DL
dl_reduced = dl_df.drop(columns=to_drop_dl)

Redundant DL features to drop: []


In [ ]:
# Remove redundant features
dl_reduced = dl_df.drop(columns=to_drop_dl)

# Define map_to_binary function
def map_to_binary(label):
    return 1 if label == 'mel' else 0

# Load labels for LDA
metadata_df_indexed = pd.read_csv(metadata_path, index_col='image_id')
labels = metadata_df_indexed.loc[common_ids, 'dx']  # Use original 'dx' (string labels)
le = LabelEncoder()  # Encode strings to ints for LDA
labels_encoded = le.fit_transform(labels)

In [14]:
# Apply LDA for reduction to maximum possible components
# Calculate the maximum number of components
n_components = len(np.unique(labels_encoded)) - 1
if n_components > 0:
    # Convert all column names to strings to avoid TypeError
    dl_reduced.columns = dl_reduced.columns.astype(str)
    
    lda = LinearDiscriminantAnalysis(n_components=n_components)
    lda_features = lda.fit_transform(dl_reduced, labels_encoded)  # Use labels_encoded for numerical labels
    lda_df = pd.DataFrame(lda_features, index=common_ids, columns=[f'lda_{i}' for i in range(n_components)])
    lda_df.to_csv(f'/kaggle/working/resnet50_finetuned_lda_{n_components}.csv')
    print(f"LDA reduced features ({n_components} components) saved to '/kaggle/working/resnet50_finetuned_lda_{n_components}.csv'")

LDA reduced features (6 components) saved to '/kaggle/working/resnet50_finetuned_lda_6.csv'
